In [2]:
import Game
from players.Player import Player
from players.RandomPlayer import RandomPlayer 
from players.DeathstackPlayer import DeathstackPlayer
from players.DeathstackPlayerV2 import DeathstackPlayerV2
from players.Pragmatic import Pragmatic
import numpy as np
import BoardEncoder as enc
from Data import adjacency

n = 100
p = 0.01
examples = []
games = []
num_players = 2
nodes = adjacency.shape[0]

rng = np.random.default_rng(0)

for i in range(n):
    players = [Pragmatic(0,seed=0), Pragmatic(1,seed=0)]
    game = Game.Game(players,seed=i)
    t = 0

    while t < 5 or rng.uniform()>p:
        game.playTurn()
        t+=1
        if game.survivingPlayers < 2:
            break

    ex = enc.getAllTrainingExamples(game)
    if ex is not None:
        games.append(game)
        examples.append([e for e in ex if enc.makeBoard(e).budget > 0])
    else:
        pass

In [3]:
xList = []
yList = []

for i in range(len(games)):
    
    xList.append(np.array(examples[i]))
    outcome = np.sum(np.arange(num_players)[:,None] == games[i].board.ownership[None,:],axis=1) / nodes
    yList.append(np.ones((len(examples[i]),num_players))*outcome)

x = np.concatenate(xList,axis=0)
y = np.concatenate(yList, axis=0)
y.shape,x.shape, y[:,0].mean()

((2017, 2), (2017, 156), 0.6537828458106099)

In [4]:
from Estimator import build as build_estimator
estimator = build_estimator(x.shape[1],num_players)
estimator.trainable = False
split = y.shape[0] //10 *9
x_train = x[:split]
y_train = y[:split]
x_test = x[split:]
y_test = y[split:]

In [5]:
estimator.fit(x_train,y_train,batch_size=32,epochs = 10,validation_data=(x_test,y_test))

Epoch 1/10

57/57 [==============================] - 2s 12ms/step - loss: 14.8687 - val_loss: 12.1710
Epoch 2/10
57/57 [==============================] - 0s 5ms/step - loss: 10.0672 - val_loss: 8.1462
Epoch 3/10
57/57 [==============================] - 0s 7ms/step - loss: 6.7579 - val_loss: 5.4972
Epoch 4/10
57/57 [==============================] - 0s 7ms/step - loss: 4.6293 - val_loss: 3.8280
Epoch 5/10
57/57 [==============================] - 0s 6ms/step - loss: 3.2587 - val_loss: 2.7094
Epoch 6/10
57/57 [==============================] - 0s 6ms/step - loss: 2.3321 - val_loss: 1.9546
Epoch 7/10
57/57 [==============================] - 0s 7ms/step - loss: 1.6948 - val_loss: 1.4306
Epoch 8/10
57/57 [==============================] - 0s 6ms/step - loss: 1.2515 - val_loss: 1.0577
Epoch 9/10
57/57 [==============================] - 0s 6ms/step - loss: 0.9369 - val_loss: 0.7983
Epoch 10/10
57/57 [==============================] - 0s 5ms/step - loss: 0.7137 - val_loss: 0.6109


In [6]:


model = build(estimator)
model.fit(x_train,batch_size=32,epochs=10)

Epoch 1/10
 8/57 [===>..........................] - ETA: 3s - eval: 0.5222

KeyboardInterrupt: 

In [2]:
import tensorflow as tf
x= tf.ones(10)
tf.range(tf.shape(x)[0])[:,None]

<tf.Tensor: shape=(10, 1), dtype=int32, numpy=
array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])>